1. Input from user
2. Local llm
3. LLM provides structured output for a function to be called
4. We call the function
5. We take the output of the function and feed that back into the llm
6. the llm provides an output for the user

In [1]:
input_prompt_from_user = """
What is Lucas profession, favorite movie and favorite book??
Use the information from the file: lucas_secrets.txt
"""

In [2]:
def get_lucas_info(file_path):
    with open(file_path, "r") as file:
        return file.read()
    
get_lucas_info(file_path="./lucas_secrets.txt")

'Name: Lucas Soares\nProfession: Software Engineer\nFavorite Movie: Inception\nFavorite Book: The Name of the Wind'

In [3]:
import ollama

response = ollama.chat(
    model='mistral-small3.2',
    messages=[{'role': 'user', 'content': 
        input_prompt_from_user}],

    tools=[{
      'type': 'function',
      'function': {
        'name': 'get_lucas_info',
        'description': 'Get the information about Lucas',
        'parameters': {
          'type': 'object',
          'properties': {
            'file_path': {
              'type': 'string',
              'description': 'The path to the file containing the information about Lucas',
            },
          },
          'required': ['file_path'],
        },
      },
    }],
)

print(response['message']['tool_calls'])

[{'function': {'name': 'get_lucas_info', 'arguments': {'file_path': 'lucas_secrets.txt'}}}]


In [4]:
def execute_tool_call(tool_call_output):
    if tool_call_output[0]['function']['name'] == 'get_lucas_info':
        return get_lucas_info(tool_call_output[0]['function']['arguments']['file_path'])
    else:
        return "Tool not found"

tool_call_output = response['message']['tool_calls']    
output_of_function = execute_tool_call(tool_call_output)
output_of_function

'Name: Lucas Soares\nProfession: Software Engineer\nFavorite Movie: Inception\nFavorite Book: The Name of the Wind'

In [5]:
new_input_with_more_information = f"""
The initial query was: {input_prompt_from_user}.
The output of the function was: {output_of_function}.
Provide the answer:
"""
response = ollama.chat(
    model='mistral-small3.2',
    messages=[{'role': 'user', 'content': 
        new_input_with_more_information}],
)

print(response)

{'model': 'mistral-small3.2', 'created_at': '2025-10-03T16:54:29.574068Z', 'message': {'role': 'assistant', 'content': 'Based on the information from the file `lucas_secrets.txt`, here are the details about Lucas:\n\n- **Profession**: Software Engineer\n- **Favorite Movie**: *Inception*\n- **Favorite Book**: *The Name of the Wind*'}, 'done_reason': 'stop', 'done': True, 'total_duration': 3948886791, 'load_duration': 67566291, 'prompt_eval_count': 579, 'prompt_eval_duration': 1046287750, 'eval_count': 52, 'eval_duration': 2833028167}


In [6]:
from IPython.display import display, Markdown

display(Markdown(f"# {output_of_function}"))

# Name: Lucas Soares
Profession: Software Engineer
Favorite Movie: Inception
Favorite Book: The Name of the Wind